3 atividade: trabalho de determinação de parâmetros exoplanetários utilizando dados de velocidaderadial e de trânsito;

Quais parametros estudar:

Determinação de massa, raio, semi eixo maior, período e outros parâmetros (se necessário) do planeta escolhido;

Estudo da zona de habitabilidade, temperatura de equilíbrio, grau de insolação do planeta (comparação com o sistema solar e com outros exoplanetas);

Estudo de densidade planetária com base no valor obtido (comparação com o sistema solar e com outros exoplanetas);

Método:

Escolha entre 2 e três planetas que possuam, necessariamente, dados de profundidade de trânsito (Transit Depth[%]), amplitude da variação da velocidade (Radial Velocity Amplitude [m/s]) e período de velocidade radial (Orbital Period [days]);

Lembrem de descrever os telescópios e satélites que fizeram as obtenções dos dados;Acesse o catálogo da NASA para fazer a seleção dos planetas:

https://exoplanetarchive.ipac.caltech.edu/cgi-bin/TblView/nph-tblView?app=ExoTbls&config=planets

https://exoplanetarchive.ipac.caltech.edu/

Em anexo está a apostila do estudo de exoplanetas. Nela vocês vão encontrar todas as equações necessárias assim como a discussão para o desenvolvimento do trabalho.

Os Planetas Escolhidos foram BD+20 594b (BD+20 594), CoRoT-2b (CoRoT-2), HAT-P-14b (HAT-P-14)

In [3]:
class SIConversions:
    def __init__(self, daysInSeconds, auInMeter):
        self.DAYS_IN_SECONDS = daysInSeconds
        self.AU_IN_METER = auInMeter
        
    def convert_DaysToSeconds(self, days):
        return days*self.DAYS_IN_SECONDS
    
    def convert_AuToMeter(self, au):
        return au*self.AU_IN_METER

In [19]:
import math

class ExoplanetParameters:
    
    def __init__(self, solTemp, albedo): 
        self.siConversions = SIConversions(86400, 149597870700)
        self.SOLAR_TEMPERATURE = solTemp
        self.ALBEDO = albedo
        
    def sunFlowRadiation(self):
        return 3.846*math.pow(10,26)/(4*math.pi*math.pow(149597870700, 2))
    
    def convertStellarRatioToMeter(self, stellarRatio):
        return stellarRatio*695500000
    
    def convertDaysToYear(self, period):
        return period/365
    
    def radiationFlow(self, stellarRatio_inSolarRatio, effectiveTemp, semiMajorAxis):
        return math.pow(stellarRatio_inSolarRatio, 2)*math.pow(effectiveTemp/self.SOLAR_TEMPERATURE, 4)*math.pow(1/semiMajorAxis, 2)
    
    def equilibriumTemperature(self, effectiveTemp, semiMajorAxis, stellarRatio):
        return effectiveTemp*math.pow(1-self.ALBEDO, 1/4)*math.pow((stellarRatio/(2*self.ALBEDO)), 1/2)
            
    def habitabilityZone(self, stellarLuminosity_inSolarLuminosity):
        intern = 0.75*math.pow(stellarLuminosity_inSolarLuminosity, 2)
        central = math.pow(stellarLuminosity_inSolarLuminosity, 2)
        extern = 1.77*math.pow(stellarLuminosity_inSolarLuminosity, 2)
        return (intern, central, extern)
    
    def planetRadius(self, transitDepth, stellarRatio_inSolarRatio):
        return math.pow(transitDepth*1.049, 1/2)*(stellarRatio_inSolarRatio)
    

In [20]:
import numpy as np

exoParams = ExoplanetParameters(5777, 0.367)
print(exoParams.habitabilityZone(2))

planet_1 = {
    "name": "BD+20 594b",
    "transitDepth": 0.0490,
    "radialVelocityAmp": 3.1,
    "orbitalPeriod": 41.688644,
    "semiMajorAxis_inAU": 0.241,
    "eccentricity": 0,
    "star":{
        "effectiveTemperature": 5651.760,
        "stellarRatio_inSolarRatio": 1.2449300,
        "stellarLuminosity": 0.15372068,
        "stellarMass_inSolarMass": 1.0030000
    }
}

planet_2 = {
    "name": "CoRoT-2b",
    "transitDepth": 2.750,
    "radialVelocityAmp": 568,
    "orbitalPeriod": 1.7429975,
    "semiMajorAxis_inAU": 0.02810,
    "eccentricity": 0,
    "star":{
        "effectiveTemperature": 5625,
        "stellarRatio_inSolarRatio": 0.902,
        "stellarLuminosity": -0.1305534,
        "stellarMass_inSolarMass": 0.970
    }
}

planet_3 = {
    "name": "HAT-P-14b",
    "transitDepth": 0.650,
    "radialVelocityAmp": 225.8,
    "orbitalPeriod": 4.6276820,
    "semiMajorAxis_inAU": 0.06062,
    "eccentricity": 0.1074,
    "star":{
        "effectiveTemperature": 6727.6100,
        "stellarRatio_inSolarRatio": 1.5452200,
        "stellarLuminosity": 0.6441211,
        "stellarMass_inSolarMass": 1.433000
    }
}

print('******************* Para o Planeta 1 *******************\n')
print("Planeta:", planet_1["name"])
print('Grau de Insolação:', exoParams.radiationFlow(planet_1["star"]["stellarRatio_inSolarRatio"], planet_1["star"]["effectiveTemperature"], planet_1["semiMajorAxis_inAU"]))
print('Temp. de Equilíbrio:', exoParams.equilibriumTemperature(planet_1["star"]["effectiveTemperature"], planet_1["semiMajorAxis_inAU"], planet_1["star"]["stellarRatio_inSolarRatio"]))
print('Raio do Planeta:', exoParams.planetRadius(planet_1["transitDepth"], planet_1["star"]["stellarRatio_inSolarRatio"]))

print('\n******************* Para o Planeta 2*******************\n')
print("Planeta:", planet_2["name"])
print('Grau de Insolação:', exoParams.radiationFlow(planet_2["star"]["stellarRatio_inSolarRatio"], planet_2["star"]["effectiveTemperature"], planet_2["semiMajorAxis_inAU"]))
print('Raio do Planeta:', exoParams.planetRadius(planet_2["transitDepth"], planet_2["star"]["stellarRatio_inSolarRatio"]))

print('\n******************* Para o Planeta 3 *******************\n')
print("Planeta:", planet_3["name"])
print('Grau de Insolação:', exoParams.radiationFlow(planet_3["star"]["stellarRatio_inSolarRatio"], planet_3["star"]["effectiveTemperature"], planet_3["semiMajorAxis_inAU"]))
print('Temp. de Equilíbrio:', exoParams.equilibriumTemperature(planet_3["star"]["effectiveTemperature"], planet_3["semiMajorAxis_inAU"], planet_3["star"]["stellarRatio_inSolarRatio"]))
print('Raio do Planeta:', exoParams.planetRadius(planet_3["transitDepth"], planet_3["star"]["stellarRatio_inSolarRatio"]))


(3.0, 4.0, 7.08)
******************* Para o Planeta 1 *******************

Planeta: BD+20 594b
Grau de Insolação: 24.444499834295595
Temp. de Equilíbrio: 6565.3680180430565
Raio do Planeta: 0.2822478982783838

******************* Para o Planeta 2*******************

Planeta: CoRoT-2b
Grau de Insolação: 926.1494480541597
Raio do Planeta: 1.5320065727665793

******************* Para o Planeta 3 *******************

Planeta: HAT-P-14b
Grau de Insolação: 1195.0402606449456
Temp. de Equilíbrio: 8706.805948638015
Raio do Planeta: 1.2759531930605998
